In [ ]:
import zipfile

# Mount the Google Drive to access the zip file
from google.colab import drive
drive.mount('/content/drive')

# Set the path to the zip file in the mounted drive
zip_path = '/content/drive/MyDrive/Dataset.zip'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt

# Get a batch of images from the training dataset
x_batch, y_batch = next(train_datagen)

# Plot the images
fig, axes = plt.subplots(nrows=4, ncols=8, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_batch[i].reshape(224, 224), cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f'Class: {y_batch[i].argmax()}')
plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model

def DFU_SPNet():
    """
    Returns a DFU_SPNet model.

    This model consists of two parallel convolutional layers that are stacked
    together, with a Concatenate layer to merge the outputs of the parallel layers,
    followed by a dense layer with dropout for classification.

    Returns:
        model (tensorflow.keras.models.Model): The DFU_SPNet model.
    """

    # Define the input layer
    input_layer = Input(shape=(224, 224, 1))

    # Define the first parallel convolutional layer
    conv1a = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    conv1a = MaxPooling2D((2, 2))(conv1a)
    conv1b = Conv2D(32, (5, 5), activation='relu', padding='same')(input_layer)
    conv1b = MaxPooling2D((2, 2))(conv1b)

    # Define the second parallel convolutional layer
    conv2a = Conv2D(64, (3, 3), activation='relu', padding='same')(Concatenate()([conv1a, conv1b]))
    conv2a = MaxPooling2D((2, 2))(conv2a)
    conv2b = Conv2D(64, (5, 5), activation='relu', padding='same')(Concatenate()([conv1a, conv1b]))
    conv2b = MaxPooling2D((2, 2))(conv2b)

    # Merge the outputs of the parallel layers using Concatenate
    merged = Concatenate()([conv2a, conv2b])

    # Flatten the output of the merged layer
    flatten = Flatten()(merged)

    # Add a dense layer with dropout for classification
    dense = Dense(128, activation='relu')(flatten)
    dense = Dropout(0.5)(dense)
    output_layer = Dense(2, activation='softmax')(dense)

    # Define the model with input and output layers
    model = Model(inputs=[input_layer], outputs=[output_layer])

    return model


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the dataset using ImageDataGenerator with data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)
train_dataset = train_datagen.flow_from_directory('/content/DFU Dataset/train',
                                                  target_size=(224, 224),
                                                  color_mode='grayscale',
                                                  batch_size=32,
                                                  class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_dataset = test_datagen.flow_from_directory('/content/DFU Dataset/test',
                                                target_size=(224, 224),
                                                color_mode='grayscale',
                                                batch_size=32,
                                                class_mode='categorical')

# Initialize the DFU_SPNet model
model = DFU_SPNet()

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset,
          epochs=50,
          validation_data=test_dataset)

In [ ]:
model.summary()

In [ ]:
history['accuracy']

In [ ]:
# Evaluate model
test_loss, test_acc = model.evaluate(test_dataset)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
# Import required libraries
import numpy as np
from tensorflow.keras.preprocessing import image

# Load an image for prediction
img_path = '/content/DFU Dataset/test/1/2.01_110.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img.show();
img_arr = image.img_to_array(img)
img_arr = np.expand_dims(img_arr, axis=0)
img_arr /= 255.

# Make prediction
pred = model.predict(img_arr)

# Print predicted class and probability
if pred[0] < 0.5:
    print('Non-ulcer foot image with probability', 1-pred[0])
else:
    print('Ulcer foot image with probability', pred[0])


In [ ]:
model_version = 1
model.save('my_model.h5')